# Notebook Model v.2 results vizualisations

## Content:
1. [Imports](#imports)
2. [Read data](#read-data)
3. [Modelling](#modelling)
4. [GUI Setup](#gui-setup)
5. [Single player GUI](#single-player-gui)
6. [Player comparison GUI](#player-comparison-gui)


## Summary


## 1. Imports <a class="anchor" id="imports"></a>

In [1]:
# Basics
import pandas as pd
import numpy as np

# Project module
import modules.viz_lib as viz
from modules.models_lib import create_PCA_scores, map_PCA_scores
from modules.config import dict_playingStyle_mapper

# gui
import ipywidgets as widgets

# Ignore warnings
import warnings
warnings.simplefilter(action="ignore")

## 2. Read data <a class="anchor" id="read-data"></a>

As inputs to the model there is the processed and filtered KPI-data found in the data directory. 
Note here that positions computed from the position detection model already exists in these imported .xlsx-files.

In [2]:
# Read model KPI dataframe from PL
df_KPI_PL = pd.read_excel('../data/model_kpis_PL21-22.xlsx')

# Read model KPI dataframe from Allsvenskan, Sweden
df_KPI_Swe = pd.read_excel('../data/model_kpis_Swe21.xlsx')

## 3. Modelling <a class="anchor" id="modelling"></a>

Use the model presented in model_v2.ipynb and result mapping as in config.py.

In [3]:
# Get PCA-scores
dict_PCA_result = create_PCA_scores(df_KPI_PL, df_KPI_Swe, screeplot=False, nr_of_PCs=6)

# Result collecting
df_result_weights = dict_PCA_result['result_weights']
df_result_PCA_PL = dict_PCA_result['result_train']
df_result_PCA_PL_excl = dict_PCA_result['result_excl_train']
df_result_PCA_Swe = dict_PCA_result['result_test']
df_result_PCA_Swe_excl = dict_PCA_result['result_excl_test']

# Map scores to playingstyles
df_playing_styles_PL = map_PCA_scores(df_result_PCA_PL, df_result_PCA_PL_excl, dict_mapper=dict_playingStyle_mapper)
df_playing_styles_Swe = map_PCA_scores(df_result_PCA_Swe, df_result_PCA_Swe_excl, dict_mapper=dict_playingStyle_mapper)

# Call to functions to get quantile scaled playing styles

## 4. GUI Setup <a class="anchor" id="gui-setup"></a>

Here all the setup for the guis are created. You can also choose here if you want to look at Premier League or Allsvenskan.


### 4.1 Inputs

In [4]:
# Choose league
league = "PL"
league = "Swe"

### 4.2 Variables

In [5]:
# Relative path to the Playmaker logo
logo_path="../figures_used/Playmaker_Logo.png"

# How many playingstyles for each position to include in plot
num_position_playingS = {'ST': 0, 'CM': 0, 'OW': 0, 'FB': 0, 'CB': 0}
num_position_playingS_comparison = {'ST': 0, 'CM': 0, 'OW': 0, 'FB': 0, 'CB': 0}

# Parameterr to use if we want to have the spider as quantile rankings in the league
use_ranking_scale = False
use_ranking_scale_comparison = False

# Find all players
list_players_PL = df_playing_styles_PL['name'].unique().tolist()
list_players_PL.sort()
list_players_Swe = df_playing_styles_Swe['name'].unique().tolist()
list_players_Swe.sort()

# set variables accordning to chosen league
if league == "Pl":
    list_players = list_players_PL
    df_playing_styles = df_playing_styles_PL
    df_KPI = df_KPI_PL
elif league == "Swe":
    list_players = list_players_Swe
    df_playing_styles = df_playing_styles_Swe
    df_KPI = df_KPI_Swe
else: 
    print("Not a valid league!")

### 4.3 Gui variables

Player lists

In [6]:
# Drop down list for single player gui
drop_down_player_list = widgets.Dropdown(
    options=list_players,
    description='Player:',
    disabled=False,
)

# Drop down list for player comparison player 1
drop_down_player_list1 = widgets.Dropdown(
    options=list_players,
    description='Player:',
    disabled=False,
)

# Drop down list for player comparison player 2
drop_down_player_list2 = widgets.Dropdown(
    options=list_players,
    description='Player:',
    disabled=False,
)

Position choices

In [7]:
bool_button_ST = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'},
            value="No",
            disabled=False
        )

bool_box_ST = widgets.Box(
    [
        widgets.Label(value='Include playing styles for position "ST"?'),
        bool_button_ST
        
    ]
)

bool_button_CM = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'},
             value="No",
            disabled=False
        )

bool_box_CM = widgets.Box(
    [
        widgets.Label(value='Include playing styles for position "CM"?'),
        bool_button_CM
    ]
)

bool_button_OW = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'},
            value="No",
            disabled=False
        )

bool_box_OW = widgets.Box(
    [
        widgets.Label(value='Include playing styles for position "OW"?'),
        bool_button_OW
        
    ]
)

bool_button_FB = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'},
             value="No",
            disabled=False
        )

bool_box_FB = widgets.Box(
    [
        widgets.Label(value='Include playing styles for position "FB"?'),
        bool_button_FB
    ]
)

bool_button_CB = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'},
            disabled=False,
            value="No",

        )

bool_box_CB = widgets.Box(
    [
        widgets.Label(value='Include playing styles for position "CB"?'),
        bool_button_CB
    ]
)

Scale choice

In [8]:
bool_quantile_scale = widgets.RadioButtons(
            options=['Yes', 'No'],
            layout={'width': 'max-content'},
            disabled=False,
            value="No",
        )

quantile_scale_box = widgets.Box(
    [
        widgets.Label(value='Use Quantile scale?'),
        bool_quantile_scale
    ]
)

Buttons and output

In [9]:
single_player_button = widgets.Button(
    description='See spider',
    disabled=False,
    button_style='', 
)
comparison_player_button = widgets.Button(
    description='See spider',
    disabled=False,
    button_style='', 
)
single_player_output = widgets.Output()
comparison_player_output = widgets.Output()

### 4.4 GUI functions

In [10]:
def handle_plot_inputs():
    global use_ranking_scale
    global num_position_playingS
    global bool_button_ST
    global bool_button_CM
    global bool_box_OW
    global bool_button_FB
    global bool_button_CB
    global bool_quantile_scale

    # Handle playing styles for positions 
    if bool_button_ST.value == 'Yes':
        num_position_playingS['ST'] = 4
    else: 
        num_position_playingS['ST'] = 0
    if bool_button_CM.value == 'Yes':
        num_position_playingS['CM'] = 3
    else: 
        num_position_playingS['CM'] = 0
    if bool_button_OW.value == 'Yes':
        num_position_playingS['OW'] = 3
    else: 
        num_position_playingS['OW'] = 0
    if bool_button_FB.value == 'Yes':
        num_position_playingS['FB'] = 3
    else: 
        num_position_playingS['FB'] = 0
    if bool_button_CB.value == 'Yes':
        num_position_playingS['CB'] = 3
    else: 
        num_position_playingS['CB'] = 0

    # Handle scale
    if bool_quantile_scale.value == 'Yes':

       use_ranking_scale = True

       # use quantile scaled playingstyles
    else: 
        use_ranking_scale = False

def see_single_player(b):
    global num_position_playingS
    global use_ranking_scale
    global df_playing_styles

    player_club = df_KPI.loc[df_KPI['name'] == drop_down_player_list.value]['club'].values[0]
    
    single_player_output.clear_output()
    handle_plot_inputs()
    with single_player_output:

        if all(value == 0 for value in num_position_playingS.values()):
            print("Choose some position(s)")
        else:
            viz.single_player_notebook_plot(df_playing_styles, drop_down_player_list.value, player_club, num_position_playingS, use_ranking_scale, logo_path)

single_player_button.on_click(see_single_player)

def compare_players(b):
    global num_position_playingS
    global use_ranking_scale
    global df_playing_styles

    player_club1 = df_KPI.loc[df_KPI['name'] == drop_down_player_list.value]['club'].values[0]
    player_club2 = df_KPI.loc[df_KPI['name'] == drop_down_player_list.value]['club'].values[0]
    
    comparison_player_output.clear_output()
    handle_plot_inputs()
    with comparison_player_output:

        if all(value == 0 for value in num_position_playingS.values()):
            print("Choose some position(s)")
        else:
            viz.compare_players_notebook_plot(df_playing_styles, drop_down_player_list1.value,  drop_down_player_list2.value, player_club1, player_club2, num_position_playingS, use_ranking_scale, logo_path)

comparison_player_button.on_click(compare_players)

## 5. Single player GUI <a class="anchor" id="single-player-gui"></a>

Scale dont work at the moment

In [11]:
widgets.HBox([widgets.VBox([drop_down_player_list, bool_box_ST, bool_box_CM, bool_box_OW, bool_box_FB, bool_box_CB, quantile_scale_box, single_player_button]), single_player_output])


## 6. Player comparison GUI <a class="anchor" id="player-comparison-gui"></a>

Scale dont work at the moment

In [12]:
widgets.HBox([widgets.VBox([drop_down_player_list1, drop_down_player_list2, bool_box_ST, bool_box_CM, bool_box_OW, bool_box_FB, bool_box_CB, quantile_scale_box, comparison_player_button]), comparison_player_output])
